Load dataset. It contains translations from English to Spanish, so swap the order of the phrases. Also add `\t` and `\n` as the start and stop tokens in the target sequences. 

In [ ]:
start_token = "\t"
stop_token = "\n"

with open("data/spa.txt", "r", encoding="utf-8") as f:
    samples = f.read().split("\n")

samples = [sample.strip().split("\t")
           for sample in samples if len(sample.strip()) > 0]

samples = [(es, start_token + en + stop_token)
           for en, es in samples if len(es) < 45]

In [ ]:
len(samples)

In [ ]:
print(samples[:2])

Split data into train and validation sets.

In [ ]:
from sklearn.model_selection import train_test_split

train_samples, valid_samples = train_test_split(samples, train_size=.8, random_state=42)

In [ ]:
len(train_samples)

In [ ]:
len(valid_samples)

Determine the training vocabulary. Those are the only tokens you can trust the model will know how to handle. 

In [ ]:
in_vocab = set()
out_vocab = set()

for in_seq, out_seq in train_samples:
    in_vocab.update(in_seq)
    out_vocab.update(out_seq)
    
in_vocab_size = len(in_vocab)
out_vocab_size = len(out_vocab)

print("Input vocab size:", in_vocab_size)
print("Output vocab size:", out_vocab_size)

In [ ]:
print(sorted(in_vocab))

In [ ]:
print(sorted(out_vocab))

Go through validation set and remove any tokens not present in the training set.

In [ ]:
tmp_samples = []
for in_seq, out_seq in valid_samples:
    tmp_in_seq = [c for c in in_seq if c in in_vocab]
    tmp_out_seq = [c for c in out_seq if c in out_vocab]

    tmp_samples.append(("".join(tmp_in_seq), "".join(tmp_out_seq)))
    
valid_samples = tmp_samples

Build sequence-to-sequence model with bidirectional encoder.

In [ ]:
import keras
from keras.layers import Concatenate, Dense, Input, GRU, Masking
from keras.models import Model

Create encoder with two GRU layers, each processing the input in a different direction

In [ ]:
latent_dim = 512

encoder_in = Input(shape=(None, in_vocab_size), name="encoder_in")
encoder_mask = Masking(name="encoder_mask")(encoder_in)

fwd_enc_gru = GRU(latent_dim, recurrent_dropout=0.3, name="fwd_enc_gru")
rev_enc_gru = GRU(latent_dim, go_backwards=True, recurrent_dropout=0.3, name="rev_enc_gru")
fwd_enc_out = fwd_enc_gru(encoder_mask)
rev_enc_out = rev_enc_gru(encoder_mask)

encoder_out = Concatenate(name="encoder_out")([fwd_enc_out, rev_enc_out])

Decoder's input dimensions are twice as big as the encoder's output dimensions because you combine the forward and reverse outputs 

In [ ]:
decoder_latent_dim = latent_dim * 2

In [ ]:
decoder_in = Input(shape=(None, out_vocab_size), name="decoder_in")

decoder_mask = Masking(name="decoder_mask")(decoder_in)
decoder_gru = GRU(decoder_latent_dim, return_sequences=True,
                  return_state=True, dropout=0.2, recurrent_dropout=0.3,
                  name="decoder_gru")
decoder_gru_out, _ = decoder_gru(decoder_mask, initial_state=encoder_out)
decoder_dense = Dense(out_vocab_size, activation="softmax", name="decoder_out")
decoder_out = decoder_dense(decoder_gru_out)

In [ ]:
seq2seq_model = Model([encoder_in, decoder_in], decoder_out)
seq2seq_model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [ ]:
seq2seq_model.summary()

Create maps to convert characters to and from ints. 

In [ ]:
in_token2int = {token : i for i, token in enumerate(sorted(in_vocab))}
out_token2int = {token : i for i, token in enumerate(sorted(out_vocab))}
out_int2token = {i : token for (token, i) in out_token2int.items()}

Create helper functions for one-hot encoding sequences for use with the model.

In [ ]:
import numpy as np

def make_batch_storage(batch_size, in_seq_len, out_seq_len):
    
    enc_in_seqs = np.zeros(
        (batch_size, in_seq_len, in_vocab_size),
        dtype=np.float32)

    dec_in_seqs = np.zeros(
        (batch_size, out_seq_len, out_vocab_size),
        dtype=np.float32)

    dec_out_seqs = np.zeros(
        (batch_size, out_seq_len, out_vocab_size),
        dtype=np.float32)
        
    return enc_in_seqs, dec_in_seqs, dec_out_seqs


In [ ]:
def encode_batch(samples):
    batch_size = len(samples)
    max_in_length = max([len(seq) for seq, _ in samples])
    max_out_length = max([len(seq) for _, seq in samples])

    enc_in_seqs, dec_in_seqs, dec_out_seqs = make_batch_storage(
        batch_size, max_in_length, max_out_length)
    
    for i, (in_seq, out_seq) in enumerate(samples):
        for time_step, token in enumerate(in_seq):
            enc_in_seqs[i, time_step, in_token2int[token]] = 1

        for time_step, token in enumerate(out_seq):
            dec_in_seqs[i, time_step, out_token2int[token]] = 1

        for time_step, token in enumerate(out_seq[1:]):
            dec_out_seqs[i, time_step, out_token2int[token]] = 1
            
    return enc_in_seqs, dec_in_seqs, dec_out_seqs

Train model

In [ ]:
from seq2seq_util import Seq2SeqBatchGenerator

batch_size = 64
train_generator = Seq2SeqBatchGenerator(train_samples, batch_size, encode_batch)
valid_generator = Seq2SeqBatchGenerator(valid_samples, batch_size, encode_batch)

### Warning:

Running the following cell can take a long time. If you are just experimenting, consider changing the `epochs` value to something small, from `1` to `10`.

In [ ]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

seq2seq_model.fit_generator(train_generator, epochs=500,
                            validation_data=valid_generator,
                            callbacks=[early_stopping])

Create encoder/decoder models for inference

In [ ]:
inf_encoder = Model(encoder_in, encoder_out)

In [ ]:
inf_encoder.summary()

In [ ]:
inf_dec_h_in = Input(shape=(decoder_latent_dim,), name="decoder_h_in")

inf_dec_gru_out, inf_dec_h_out = decoder_gru(
    decoder_in, initial_state=inf_dec_h_in)

inf_dec_out = decoder_dense(inf_dec_gru_out)

inf_decoder = Model(
    [decoder_in, inf_dec_h_in],
    [inf_dec_out, inf_dec_h_out])

In [ ]:
inf_decoder.summary()

Test trained model on the first 100 samples from both the training and validation sets.

In [ ]:
max_out_seq_len = max([len(seq) for _, seq in samples])
print("Max output length: ", max_out_seq_len)

start_token_idx = out_token2int[start_token]
stop_token_idx = out_token2int[stop_token]

In [ ]:
def translate_sequence(one_hot_seq, encoder, decoder):
    encoding = encoder.predict(one_hot_seq)
    
    decoder_in = np.zeros((1, 1, out_vocab_size), dtype=np.float32)

    translated_text = ''
    done_decoding = False
    decoded_idx = start_token_idx
    while not done_decoding:
        decoder_in[0, 0, decoded_idx] = 1
        decoding, encoding = decoder.predict([decoder_in, encoding])
        decoder_in[0, 0, decoded_idx] = 0

        decoded_idx = np.argmax(decoding[0, -1, :])
        
        if decoded_idx == stop_token_idx:
            done_decoding = True
        else:
            translated_text += out_int2token[decoded_idx]

        if len(translated_text) >= max_out_seq_len:
            done_decoding = True
            
    return translated_text

Validation samples:

In [ ]:
from seq2seq_util import test_predictions

test_predictions(valid_samples[:100], inf_encoder, inf_decoder, encode_batch, translate_sequence)

Training samples:

In [ ]:
test_predictions(train_samples[:100], inf_encoder, inf_decoder, encode_batch, translate_sequence)

Export model in Core ML format.

In [ ]:
coreml_enc_in = Input(shape=(None, in_vocab_size), name="encoder_in")

coreml_fwd_enc_gru = GRU(latent_dim, name="fwd_enc_gru")
coreml_rev_enc_gru = GRU(latent_dim, go_backwards=True, name="rev_enc_gru")

coreml_fwd_out = coreml_fwd_enc_gru(coreml_enc_in)
coreml_rev_out = coreml_rev_enc_gru(coreml_enc_in)

coreml_enc_out = Concatenate(name="encoder_out")([coreml_fwd_out, coreml_rev_out])

coreml_encoder_model = Model(coreml_enc_in, coreml_enc_out)
coreml_encoder_model.output_layers = coreml_encoder_model._output_layers

inf_encoder.save_weights("Es2EnBidirGruCharEncoderWeights.h5")
coreml_encoder_model.load_weights("Es2EnBidirGruCharEncoderWeights.h5")

In [ ]:
import coremltools

coreml_encoder = coremltools.converters.keras.convert(
    coreml_encoder_model,
    input_names="oneHotEncodedSeq",
    output_names="decodersIntialState")

coreml_encoder.save("Es2EnBidirGruCharEncoder.mlmodel")

In [ ]:
coreml_dec_in = Input(shape=(None, out_vocab_size))

coreml_dec_gru = GRU(decoder_latent_dim, return_sequences=True, return_state=True, name="decoder_gru")
coreml_dec_gru_out, _ = coreml_dec_gru(coreml_dec_in)
coreml_dec_dense = Dense(out_vocab_size, activation="softmax")
coreml_dec_out = coreml_dec_dense(coreml_dec_gru_out)

coreml_decoder_model = Model(coreml_dec_in, coreml_dec_out)
coreml_decoder_model.output_layers = coreml_decoder_model._output_layers

inf_decoder.save_weights("Es2EnBidirGruCharDecoderWeights.h5")
coreml_decoder_model.load_weights("Es2EnBidirGruCharDecoderWeights.h5")

In [ ]:
coreml_decoder = coremltools.converters.keras.convert(
    coreml_decoder_model,
    input_names="encodedChar",
    output_names="nextCharProbs")

coreml_decoder.save("Es2EnBidirGruCharDecoder.mlmodel")

Convert weights to 16bit floats. This shouldn't hurt performance much, if at all, and it reduces the app's download size.

In [ ]:
def convert_to_fp16(mlmodel_filename):
    basename = mlmodel_filename[:-len(".mlmodel")]
    spec = coremltools.utils.load_spec(mlmodel_filename)
    spec_16bit = \
      coremltools.utils.convert_neural_network_spec_weights_to_fp16(spec)
    coremltools.utils.save_spec(spec_16bit, f"{basename}16Bit.mlmodel")

In [ ]:
convert_to_fp16("Es2EnBidirGruCharEncoder.mlmodel")
convert_to_fp16("Es2EnBidirGruCharDecoder.mlmodel")

In [ ]:
import json

with open("esCharToInt.json", "w") as f:
    json.dump(in_token2int, f)
with open("intToEnChar.json", "w") as f:
    json.dump(out_int2token, f)